# Creating a Custom Rate

Let's imagine wanting to add a rate that has a temperature form not known to pynucastro.  We can accomplish this by creating a new class derived from `Rate`.

We'll consider a rate of the form:

$$r = r_0 \left (\frac{T}{T_0} \right )^\nu$$

representing a reaction of the form

$$A + B \rightarrow C + D$$

then we expect the $\dot{Y}$ evolution equation for this rate to have the form:

$$\frac{dY(A)}{dt} = -\rho Y(A) Y(B) r_0 \left ( \frac{T}{T_0} \right )^\nu$$

and likewise for the other nuclei.


In [1]:
import pynucastro as pyna

We'll use this to approximate the rate ${}^{14}\mathrm{N}(p, \gamma){}^{15}\mathrm{O}$ around a temperature of
$T = 3\times 10^7~\mathrm{K}$

In [2]:
rl = pyna.ReacLibLibrary()
r = rl.get_rate_by_name("n14(p,g)o15")

We can get the values of $r_0$ and $\nu$ about this temperature from the rate

In [3]:
T0 = 3.e7
nu = r.get_rate_exponent(T0)
r0 = r.eval(T0)
print(r0, nu)

1.416655077954945e-13 15.601859314950396


Now we can write our custom rate

In [19]:
class MyRate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):
        super().__init__(reactants=reactants, products=products)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"
    
    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        fstring = ""
        fstring += "@numba.njit()\n"
        fstring += f"def {self.fname}(rate_eval, tf):\n"
        fstring += f"    rate_eval.{self.fname} = {self.r0} * (tfactors.T9 * 1.e9 / {self.T0} )**({self.nu})\n\n"
        return fstring

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**nu

In [20]:
r_custom = MyRate(reactants=[pyna.Nucleus("n14"), pyna.Nucleus("p")],
                  products=[pyna.Nucleus("o15")],
                  r0=r0, T0=T0, nu=nu)

In [21]:
r_custom.fname

'n14_p__o15__generic'

In [22]:
print(r_custom.function_string_py())

@numba.njit()
def n14_p__o15__generic(rate_eval, tf):
    rate_eval.n14_p__o15__generic = 1.416655077954945e-13 * (tfactors.T9 * 1.e9 / 30000000.0 )**(15.601859314950396)




## Creating a network with our rate

Now let's create a network that includes this rate.  We'll base it off of the CNO net, but we'll leave out the rate that we are approximating.

In [23]:
rate_names = ["c12(p,g)n13",
              "c13(p,g)n14",
              "n13(,)c13",
              "n13(p,g)o14",
              "n15(p,a)c12",
              "o14(,)n14",
              "o15(,)n15"]
rates = rl.get_rate_by_name(rate_names)

In [24]:
pynet = pyna.PythonNetwork(rates=rates+[r_custom])

In [25]:
pynet.write_network()

import numba
import numpy as np
from numba.experimental import jitclass

from pynucastro.rates import TableIndex, TableInterpolator, TabularRate, Tfactors
from pynucastro.screening import PlasmaState, ScreenFactors

jp = 0
jhe4 = 1
jc12 = 2
jc13 = 3
jn13 = 4
jn14 = 5
jn15 = 6
jo14 = 7
jo15 = 8
nnuc = 9

A = np.zeros((nnuc), dtype=np.int32)

A[jp] = 1
A[jhe4] = 4
A[jc12] = 12
A[jc13] = 13
A[jn13] = 13
A[jn14] = 14
A[jn15] = 15
A[jo14] = 14
A[jo15] = 15

Z = np.zeros((nnuc), dtype=np.int32)

Z[jp] = 1
Z[jhe4] = 2
Z[jc12] = 6
Z[jc13] = 6
Z[jn13] = 7
Z[jn14] = 7
Z[jn15] = 7
Z[jo14] = 8
Z[jo15] = 8

names = []
names.append("h1")
names.append("he4")
names.append("c12")
names.append("c13")
names.append("n13")
names.append("n14")
names.append("n15")
names.append("o14")
names.append("o15")

def to_composition(Y):
    """Convert an array of molar fractions to a Composition object."""
    from pynucastro import Composition, Nucleus
    nuclei = [Nucleus.from_cache(name) for name in names]
    com